# Phase 5 — User Explanation Layer

This notebook tests and debugs the user explanation layer:
- Highlight why misleading: e.g. clickbait headline, missing context
- Show manipulation cues chips (fear appeal / cherry-picking)
- Generate prebunk tip card (20s read) to educate user

## Step 1: Setup and Dependencies

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

import json
import re
from typing import List, Dict, Tuple, Optional
from enum import Enum
import random
from datetime import datetime

# Test data from Phase 4
test_phase4_output = {
    'claim': 'The new COVID vaccine causes severe side effects in 80% of patients',
    'verdict': 'REFUTED',
    'confidence': 0.85,
    'confidence_badge': {'emoji': '🔴', 'label': 'Likely False', 'color': 'red'},
    'certainty_level': 'High',
    'citations': [
        {
            'id': 'citation_1',
            'display_title': '❌ PIB Fact Check: No Evidence of 80% Severe Side Effects from COVID Vaccines',
            'source': 'Press Information Bureau (PIB)',
            'stance': 'REFUTED',
            'highlighted_snippet': 'Clinical trials show **SIDE** **EFFECTS** occur in less than 5% of **PATIENTS** and are mostly mild...'
        },
        {
            'id': 'citation_2', 
            'display_title': '❌ WHO Updates COVID-19 Vaccine Safety Guidelines',
            'source': 'World Health Organization',
            'stance': 'REFUTED',
            'highlighted_snippet': 'Studies confirm **VACCINES** are safe with rare serious adverse events...'
        }
    ],
    'evidence_summary': {
        'total_sources': 2,
        'stance_distribution': {'REFUTED': 2},
        'avg_credibility': 0.95
    }
}

print("Dependencies loaded successfully!")
print(f"Test claim: {test_phase4_output['claim']}")
print(f"Verdict: {test_phase4_output['verdict']} ({test_phase4_output['confidence']:.2f} confidence)")

## Step 2: Manipulation Cues Detection

In [ ]:
class ManipulationCue(Enum):
    FEAR_APPEAL = "fear_appeal"
    CHERRY_PICKING = "cherry_picking"
    FALSE_AUTHORITY = "false_authority"
    EMOTIONAL_LANGUAGE = "emotional_language"
    SPECIFIC_NUMBERS = "specific_numbers"
    URGENCY = "urgency"
    CONSPIRACY = "conspiracy"
    BANDWAGON = "bandwagon"
    OVERSIMPLIFICATION = "oversimplification"

def detect_manipulation_cues(claim: str) -> List[Dict[str, any]]:
    """Detect manipulation techniques used in the claim"""
    detected_cues = []
    claim_lower = claim.lower()
    
    # Fear appeal detection
    fear_keywords = [
        'dangerous', 'deadly', 'harmful', 'toxic', 'severe', 'serious',
        'devastating', 'catastrophic', 'alarming', 'shocking', 'terrifying'
    ]
    fear_matches = [word for word in fear_keywords if word in claim_lower]
    if fear_matches:
        detected_cues.append({
            'type': ManipulationCue.FEAR_APPEAL.value,
            'confidence': min(len(fear_matches) * 0.3, 1.0),
            'evidence': fear_matches,
            'description': 'Uses fear-inducing language to provoke emotional response'
        })
    
    # Specific numbers (often used to appear authoritative)
    number_pattern = r'\b\d+(?:\.\d+)?\s*%|\b\d+\s*percent|\b\d+\s*out\s*of\s*\d+'
    number_matches = re.findall(number_pattern, claim)
    if number_matches:
        detected_cues.append({
            'type': ManipulationCue.SPECIFIC_NUMBERS.value,
            'confidence': 0.7,
            'evidence': number_matches,
            'description': 'Uses specific statistics without credible source'
        })
    
    # Emotional language
    emotional_keywords = [
        'shocking', 'unbelievable', 'amazing', 'incredible', 'outrageous',
        'scandalous', 'miraculous', 'breakthrough', 'revolutionary'
    ]
    emotional_matches = [word for word in emotional_keywords if word in claim_lower]
    if emotional_matches:
        detected_cues.append({
            'type': ManipulationCue.EMOTIONAL_LANGUAGE.value,
            'confidence': min(len(emotional_matches) * 0.4, 1.0),
            'evidence': emotional_matches,
            'description': 'Uses emotional language to bypass critical thinking'
        })
    
    # False authority
    vague_authority = [
        'experts say', 'doctors warn', 'scientists claim', 'studies show',
        'research proves', 'officials confirm'
    ]
    authority_matches = [phrase for phrase in vague_authority if phrase in claim_lower]
    if authority_matches:
        detected_cues.append({
            'type': ManipulationCue.FALSE_AUTHORITY.value,
            'confidence': min(len(authority_matches) * 0.5, 1.0),
            'evidence': authority_matches,
            'description': 'References vague authorities without specific citations'
        })
    
    # Conspiracy indicators
    conspiracy_keywords = [
        'hiding', 'cover up', 'secret', 'conspiracy', 'they don\'t want you to know',
        'suppressed', 'censored', 'banned', 'forbidden'
    ]
    conspiracy_matches = [word for word in conspiracy_keywords if word in claim_lower]
    if conspiracy_matches:
        detected_cues.append({
            'type': ManipulationCue.CONSPIRACY.value,
            'confidence': min(len(conspiracy_matches) * 0.6, 1.0),
            'evidence': conspiracy_matches,
            'description': 'Suggests conspiracy or cover-up without evidence'
        })
    
    # Urgency
    urgency_keywords = [
        'immediately', 'urgent', 'breaking', 'emergency', 'critical',
        'must act now', 'time running out', 'before it\'s too late'
    ]
    urgency_matches = [word for word in urgency_keywords if word in claim_lower]
    if urgency_matches:
        detected_cues.append({
            'type': ManipulationCue.URGENCY.value,
            'confidence': min(len(urgency_matches) * 0.4, 1.0),
            'evidence': urgency_matches,
            'description': 'Creates false sense of urgency to prevent critical evaluation'
        })
    
    return detected_cues

# Test manipulation cue detection
test_claims = [
    "The new COVID vaccine causes severe side effects in 80% of patients",
    "SHOCKING: Doctors don't want you to know this DEADLY secret about vaccines!",
    "BREAKING: Scientists discover miracle cure but government is hiding it!",
    "Studies show vaccines are safe and effective for most people"
]

print("Manipulation cue detection results:")
for claim in test_claims:
    cues = detect_manipulation_cues(claim)
    print(f"\nClaim: {claim}")
    print(f"Manipulation cues found: {len(cues)}")
    for cue in cues:
        print(f"  - {cue['type']}: {cue['description']} (confidence: {cue['confidence']:.2f})")
        print(f"    Evidence: {cue['evidence']}")

## Step 3: Misleading Context Analysis

In [ ]:
def analyze_misleading_context(claim: str, verdict: str, citations: List[Dict]) -> Dict[str, any]:
    """Analyze why the claim is misleading based on evidence"""
    misleading_factors = []
    
    # Factor 1: Statistical manipulation
    if '80%' in claim and verdict == 'REFUTED':
        # Look for actual statistics in citations
        actual_stats = []
        for citation in citations:
            snippet = citation.get('highlighted_snippet', '')
            if '5%' in snippet or 'less than' in snippet:
                actual_stats.append('less than 5%')
        
        if actual_stats:
            misleading_factors.append({
                'type': 'statistical_exaggeration',
                'severity': 'high',
                'description': f"Claim states 80% but evidence shows {actual_stats[0]}",
                'explanation': "The claim grossly exaggerates the actual rate by a factor of 16x"
            })
    
    # Factor 2: Missing context
    context_indicators = [
        ('side effects', 'mild', 'Claim omits that most side effects are mild'),
        ('vaccine', 'safe', 'Claim ignores overall safety profile'),
        ('patients', 'clinical trials', 'Claim doesn\'t specify the study context')
    ]
    
    for claim_term, context_term, explanation in context_indicators:
        if claim_term in claim.lower():
            # Check if context is provided in citations
            context_found = any(context_term in citation.get('highlighted_snippet', '').lower() 
                              for citation in citations)
            if context_found:
                misleading_factors.append({
                    'type': 'missing_context',
                    'severity': 'medium',
                    'description': explanation,
                    'explanation': f"Important context about '{context_term}' is missing from the claim"
                })
    
    # Factor 3: Source credibility issues
    if not any('government' in citation.get('source', '').lower() or 
               'who' in citation.get('source', '').lower() or
               'official' in citation.get('source', '').lower()
               for citation in citations):
        misleading_factors.append({
            'type': 'lack_of_authoritative_source',
            'severity': 'medium',
            'description': 'Claim lacks backing from authoritative health sources',
            'explanation': 'No credible health authorities support this claim'
        })
    
    # Factor 4: Contradicts scientific consensus
    if verdict == 'REFUTED' and len(citations) >= 2:
        all_refute = all(citation.get('stance') == 'REFUTED' for citation in citations)
        if all_refute:
            misleading_factors.append({
                'type': 'contradicts_consensus',
                'severity': 'high',
                'description': 'Claim directly contradicts scientific consensus',
                'explanation': f"Multiple authoritative sources ({len(citations)}) contradict this claim"
            })
    
    return {
        'misleading_factors': misleading_factors,
        'severity_score': sum(1 if f['severity'] == 'high' else 0.5 for f in misleading_factors),
        'total_factors': len(misleading_factors)
    }

# Test misleading context analysis
misleading_analysis = analyze_misleading_context(
    test_phase4_output['claim'],
    test_phase4_output['verdict'],
    test_phase4_output['citations']
)

print("Misleading context analysis:")
print(f"Total misleading factors: {misleading_analysis['total_factors']}")
print(f"Severity score: {misleading_analysis['severity_score']:.1f}")
print("\nMisleading factors detected:")

for factor in misleading_analysis['misleading_factors']:
    severity_emoji = '🔴' if factor['severity'] == 'high' else '🟡'
    print(f"  {severity_emoji} {factor['type'].replace('_', ' ').title()}")
    print(f"    Description: {factor['description']}")
    print(f"    Explanation: {factor['explanation']}")
    print()

## Step 4: Manipulation Cue Chips Generation

In [ ]:
def generate_manipulation_chips(manipulation_cues: List[Dict]) -> List[Dict[str, any]]:
    """Generate user-friendly manipulation cue chips"""
    chip_templates = {
        ManipulationCue.FEAR_APPEAL.value: {
            'emoji': '😰',
            'title': 'Fear Appeal',
            'short_description': 'Uses scary language',
            'explanation': 'This content uses fear-inducing words to make you feel anxious and bypass critical thinking.',
            'tip': 'Ask yourself: Are the fears justified by evidence?'
        },
        ManipulationCue.SPECIFIC_NUMBERS.value: {
            'emoji': '📊',
            'title': 'Questionable Statistics',
            'short_description': 'Precise numbers without source',
            'explanation': 'Specific percentages or numbers can seem authoritative but may lack credible sources.',
            'tip': 'Always ask: Where do these numbers come from?'
        },
        ManipulationCue.EMOTIONAL_LANGUAGE.value: {
            'emoji': '💢',
            'title': 'Emotional Language',
            'short_description': 'Designed to trigger emotions',
            'explanation': 'Strong emotional words are used to provoke reactions rather than encourage thinking.',
            'tip': 'Take a breath and focus on facts, not feelings.'
        },
        ManipulationCue.FALSE_AUTHORITY.value: {
            'emoji': '👨‍⚕️',
            'title': 'Vague Authority',
            'short_description': 'Claims expert support without names',
            'explanation': 'References to "doctors" or "experts" without specific names or credentials.',
            'tip': 'Look for specific, named, and qualified sources.'
        },
        ManipulationCue.CONSPIRACY.value: {
            'emoji': '🕵️',
            'title': 'Conspiracy Thinking',
            'short_description': 'Suggests hidden agenda',
            'explanation': 'Implies that information is being hidden or suppressed without providing evidence.',
            'tip': 'Extraordinary claims require extraordinary evidence.'
        },
        ManipulationCue.URGENCY.value: {
            'emoji': '⏰',
            'title': 'False Urgency',
            'short_description': 'Pressures quick action',
            'explanation': 'Creates time pressure to prevent you from fact-checking or thinking critically.',
            'tip': 'Important decisions deserve time for verification.'
        }
    }
    
    chips = []
    for cue in manipulation_cues:
        cue_type = cue['type']
        if cue_type in chip_templates:
            template = chip_templates[cue_type]
            chip = {
                'id': f"chip_{cue_type}",
                'emoji': template['emoji'],
                'title': template['title'],
                'short_description': template['short_description'],
                'explanation': template['explanation'],
                'tip': template['tip'],
                'confidence': cue['confidence'],
                'evidence': cue['evidence'],
                'severity': 'high' if cue['confidence'] > 0.7 else 'medium' if cue['confidence'] > 0.4 else 'low'
            }
            chips.append(chip)
    
    # Sort by confidence (most confident first)
    chips.sort(key=lambda x: x['confidence'], reverse=True)
    
    return chips

def format_chips_for_display(chips: List[Dict]) -> Dict[str, any]:
    """Format chips for user interface display"""
    if not chips:
        return {
            'count': 0,
            'chips': [],
            'overall_manipulation_score': 0.0
        }
    
    # Calculate overall manipulation score
    overall_score = sum(chip['confidence'] for chip in chips) / len(chips)
    
    # Format for display
    formatted_chips = []
    for chip in chips:
        formatted_chip = {
            'id': chip['id'],
            'display_text': f"{chip['emoji']} {chip['title']}",
            'tooltip': chip['short_description'],
            'full_explanation': chip['explanation'],
            'action_tip': chip['tip'],
            'severity': chip['severity'],
            'evidence_count': len(chip['evidence']) if isinstance(chip['evidence'], list) else 1
        }
        formatted_chips.append(formatted_chip)
    
    return {
        'count': len(chips),
        'chips': formatted_chips,
        'overall_manipulation_score': overall_score,
        'manipulation_level': 'high' if overall_score > 0.7 else 'medium' if overall_score > 0.4 else 'low'
    }

# Test manipulation chip generation
manipulation_cues = detect_manipulation_cues(test_phase4_output['claim'])
manipulation_chips = generate_manipulation_chips(manipulation_cues)
formatted_chips = format_chips_for_display(manipulation_chips)

print("Manipulation cue chips:")
print(f"Total chips: {formatted_chips['count']}")
print(f"Overall manipulation level: {formatted_chips['manipulation_level']}")
print(f"Manipulation score: {formatted_chips['overall_manipulation_score']:.2f}")
print("\nGenerated chips:")

for chip in formatted_chips['chips']:
    severity_indicator = '🔴' if chip['severity'] == 'high' else '🟡' if chip['severity'] == 'medium' else '🟢'
    print(f"\n{severity_indicator} {chip['display_text']}")
    print(f"   Tooltip: {chip['tooltip']}")
    print(f"   Explanation: {chip['full_explanation']}")
    print(f"   Tip: {chip['action_tip']}")
    print(f"   Evidence count: {chip['evidence_count']}")

## Step 5: Prebunk Tip Card Generation

In [ ]:
def generate_prebunk_tip_card(claim: str, verdict: str, manipulation_chips: List[Dict], misleading_analysis: Dict) -> Dict[str, any]:
    """Generate educational prebunk tip card (20 second read)"""
    
    # Determine card theme based on verdict and manipulation level
    if verdict == 'REFUTED':
        card_theme = 'false_claim'
        primary_color = '#ff4444'
        icon = '❌'
    elif verdict == 'SUPPORTED':
        card_theme = 'accurate_claim'
        primary_color = '#44ff44'
        icon = '✅'
    else:
        card_theme = 'unclear_claim'
        primary_color = '#ffaa00'
        icon = '❓'
    
    # Generate main takeaway
    if verdict == 'REFUTED':
        main_takeaway = "This claim contains false information that contradicts credible sources."
    elif verdict == 'SUPPORTED':
        main_takeaway = "This claim is supported by credible evidence and sources."
    else:
        main_takeaway = "This claim needs more evidence for a definitive conclusion."
    
    # Generate key learning points (max 3 for 20-second read)
    learning_points = []
    
    # Point 1: Most important manipulation technique
    if manipulation_chips:
        top_chip = manipulation_chips[0]  # Highest confidence
        learning_points.append({
            'icon': top_chip['emoji'],
            'title': f"Watch for {top_chip['title']}",
            'description': top_chip['tip']
        })
    
    # Point 2: Source verification
    learning_points.append({
        'icon': '🔍',
        'title': 'Check the Source',
        'description': 'Look for specific, named experts and official health organizations.'
    })
    
    # Point 3: Statistical awareness
    if any('statistical' in factor['type'] for factor in misleading_analysis.get('misleading_factors', [])):
        learning_points.append({
            'icon': '📊',
            'title': 'Question Statistics',
            'description': 'Ask where numbers come from and if they\'re from credible studies.'
        })
    else:
        learning_points.append({
            'icon': '🧠',
            'title': 'Think Critically',
            'description': 'Pause before sharing. Does this claim sound too extreme?'
        })
    
    # Generate action steps
    action_steps = [
        "Verify with official health sources (WHO, CDC, government health departments)",
        "Look for peer-reviewed studies and multiple confirming sources",
        "Be skeptical of claims with extreme statistics or emotional language"
    ]
    
    # Generate related topics for further learning
    related_topics = []
    if 'vaccine' in claim.lower():
        related_topics = ['Vaccine Safety', 'Clinical Trials', 'Adverse Event Reporting']
    elif 'covid' in claim.lower():
        related_topics = ['COVID-19 Facts', 'Pandemic Response', 'Health Misinformation']
    else:
        related_topics = ['Fact-Checking', 'Source Evaluation', 'Critical Thinking']
    
    # Calculate reading time (average 200 words per minute)
    word_count = len(main_takeaway.split()) + sum(len(lp['description'].split()) for lp in learning_points)
    reading_time = max(15, min(25, (word_count / 200) * 60))  # 15-25 seconds
    
    return {
        'id': f"prebunk_card_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        'theme': card_theme,
        'icon': icon,
        'primary_color': primary_color,
        'title': f"Fact-Check: {verdict.title()} Claim",
        'main_takeaway': main_takeaway,
        'learning_points': learning_points,
        'action_steps': action_steps,
        'related_topics': related_topics,
        'reading_time_seconds': int(reading_time),
        'created_at': datetime.now().isoformat(),
        'claim_snippet': claim[:80] + '...' if len(claim) > 80 else claim
    }

# Test prebunk tip card generation
prebunk_card = generate_prebunk_tip_card(
    test_phase4_output['claim'],
    test_phase4_output['verdict'],
    manipulation_chips,
    misleading_analysis
)

print("Generated Prebunk Tip Card:")
print(f"\n{prebunk_card['icon']} {prebunk_card['title']}")
print(f"Reading time: {prebunk_card['reading_time_seconds']} seconds")
print(f"\nMain takeaway:")
print(f"  {prebunk_card['main_takeaway']}")

print(f"\nKey learning points:")
for i, point in enumerate(prebunk_card['learning_points'], 1):
    print(f"  {i}. {point['icon']} {point['title']}")
    print(f"     {point['description']}")

print(f"\nAction steps:")
for i, step in enumerate(prebunk_card['action_steps'], 1):
    print(f"  {i}. {step}")

print(f"\nRelated topics: {', '.join(prebunk_card['related_topics'])}")
print(f"\nClaim snippet: {prebunk_card['claim_snippet']}")

## Step 6: Educational Content Personalization

In [ ]:
def personalize_educational_content(user_profile: Dict, prebunk_card: Dict) -> Dict[str, any]:
    """Personalize educational content based on user profile"""
    
    # Default user profile for demo
    default_profile = {
        'age_group': 'adult',  # young, adult, senior
        'education_level': 'college',  # high_school, college, graduate
        'health_literacy': 'medium',  # low, medium, high
        'preferred_language': 'english',
        'misinformation_vulnerability': 'medium',  # low, medium, high
        'previous_interactions': 3
    }
    
    profile = {**default_profile, **user_profile}
    
    # Adjust content complexity
    if profile['education_level'] == 'high_school' or profile['health_literacy'] == 'low':
        complexity_level = 'simple'
        vocab_adjustment = 'basic'
    elif profile['education_level'] == 'graduate' and profile['health_literacy'] == 'high':
        complexity_level = 'advanced'
        vocab_adjustment = 'technical'
    else:
        complexity_level = 'moderate'
        vocab_adjustment = 'standard'
    
    # Adjust explanation depth
    if profile['misinformation_vulnerability'] == 'high':
        explanation_depth = 'detailed'
        include_inoculation = True
    else:
        explanation_depth = 'concise'
        include_inoculation = False
    
    # Personalized learning points
    personalized_points = []
    base_points = prebunk_card['learning_points']
    
    for point in base_points:
        personalized_point = point.copy()
        
        # Adjust language complexity
        if vocab_adjustment == 'basic':
            # Simplify technical terms
            personalized_point['description'] = personalized_point['description'].replace(
                'peer-reviewed studies', 'official research'
            ).replace(
                'credible studies', 'trustworthy research'
            )
        elif vocab_adjustment == 'technical':
            # Add more technical detail
            if 'statistics' in personalized_point['description'].lower():
                personalized_point['description'] += ' Check methodology and sample size.'
        
        personalized_points.append(personalized_point)
    
    # Add inoculation content if needed
    inoculation_content = None
    if include_inoculation:
        inoculation_content = {
            'title': 'Be Prepared for Similar Claims',
            'description': 'You might see similar false claims about vaccines or health treatments. Use the same critical thinking skills.',
            'warning_signs': [
                'Extreme percentages without sources',
                'Claims that "authorities don\'t want you to know"',
                'Emotional language designed to scare'
            ]
        }
    
    # Adjust reading time based on complexity
    base_time = prebunk_card['reading_time_seconds']
    if complexity_level == 'simple':
        adjusted_time = int(base_time * 0.8)  # Faster read
    elif complexity_level == 'advanced':
        adjusted_time = int(base_time * 1.3)  # Slower read
    else:
        adjusted_time = base_time
    
    return {
        'personalized_card': {
            **prebunk_card,
            'learning_points': personalized_points,
            'reading_time_seconds': adjusted_time,
            'complexity_level': complexity_level,
            'explanation_depth': explanation_depth,
            'inoculation_content': inoculation_content
        },
        'personalization_metadata': {
            'user_profile': profile,
            'adjustments_made': {
                'complexity_level': complexity_level,
                'vocab_adjustment': vocab_adjustment,
                'explanation_depth': explanation_depth,
                'include_inoculation': include_inoculation
            }
        }
    }

# Test personalization for different user types
user_profiles = [
    {
        'name': 'High School Student',
        'education_level': 'high_school',
        'health_literacy': 'low',
        'misinformation_vulnerability': 'high'
    },
    {
        'name': 'College Graduate',
        'education_level': 'college',
        'health_literacy': 'medium',
        'misinformation_vulnerability': 'medium'
    },
    {
        'name': 'Healthcare Professional',
        'education_level': 'graduate',
        'health_literacy': 'high',
        'misinformation_vulnerability': 'low'
    }
]

print("Personalized educational content:")
for profile in user_profiles:
    personalized = personalize_educational_content(profile, prebunk_card)
    card = personalized['personalized_card']
    meta = personalized['personalization_metadata']
    
    print(f"\n=== {profile['name']} ===")
    print(f"Complexity level: {card['complexity_level']}")
    print(f"Reading time: {card['reading_time_seconds']} seconds")
    print(f"Vocab adjustment: {meta['adjustments_made']['vocab_adjustment']}")
    print(f"Include inoculation: {meta['adjustments_made']['include_inoculation']}")
    
    if card['inoculation_content']:
        print(f"Inoculation content: {card['inoculation_content']['title']}")
    
    # Show first learning point to see personalization
    if card['learning_points']:
        first_point = card['learning_points'][0]
        print(f"Sample learning point: {first_point['title']} - {first_point['description'][:60]}...")

## Step 7: Complete Phase 5 Pipeline

In [ ]:
def phase5_pipeline(phase4_output: Dict, user_profile: Optional[Dict] = None) -> Dict[str, any]:
    """Complete Phase 5 pipeline: User Explanation Layer"""
    pipeline_result = {
        'phase': 'Phase 5 - User Explanation Layer',
        'input_claim': phase4_output['claim'],
        'steps': []
    }
    
    try:
        claim = phase4_output['claim']
        verdict = phase4_output['verdict']
        citations = phase4_output['citations']
        
        # Step 1: Detect manipulation cues
        manipulation_cues = detect_manipulation_cues(claim)
        pipeline_result['steps'].append({
            'step': 'manipulation_detection',
            'result': {
                'cues_detected': len(manipulation_cues),
                'cue_types': [cue['type'] for cue in manipulation_cues]
            }
        })
        
        # Step 2: Analyze misleading context
        misleading_analysis = analyze_misleading_context(claim, verdict, citations)
        pipeline_result['steps'].append({
            'step': 'misleading_analysis',
            'result': misleading_analysis
        })
        
        # Step 3: Generate manipulation chips
        manipulation_chips = generate_manipulation_chips(manipulation_cues)
        formatted_chips = format_chips_for_display(manipulation_chips)
        pipeline_result['steps'].append({
            'step': 'manipulation_chips',
            'result': formatted_chips
        })
        
        # Step 4: Generate prebunk tip card
        prebunk_card = generate_prebunk_tip_card(claim, verdict, manipulation_chips, misleading_analysis)
        pipeline_result['steps'].append({
            'step': 'prebunk_card_generation',
            'result': prebunk_card
        })
        
        # Step 5: Personalize content (if user profile provided)
        if user_profile:
            personalized_content = personalize_educational_content(user_profile, prebunk_card)
            final_card = personalized_content['personalized_card']
            personalization_meta = personalized_content['personalization_metadata']
        else:
            final_card = prebunk_card
            personalization_meta = None
        
        pipeline_result['steps'].append({
            'step': 'content_personalization',
            'result': {
                'personalized': user_profile is not None,
                'personalization_metadata': personalization_meta
            }
        })
        
        # Final output
        pipeline_result['final_output'] = {
            'claim': claim,
            'verdict_summary': f"{phase4_output['confidence_badge']['emoji']} {verdict} ({phase4_output['certainty_level']} Confidence)",
            'manipulation_chips': formatted_chips,
            'misleading_factors': misleading_analysis['misleading_factors'],
            'prebunk_card': final_card,
            'education_summary': {
                'manipulation_level': formatted_chips['manipulation_level'],
                'learning_points_count': len(final_card['learning_points']),
                'reading_time': final_card['reading_time_seconds'],
                'personalized': user_profile is not None
            },
            'ready_for_phase6': True
        }
        pipeline_result['status'] = 'success'
        
    except Exception as e:
        pipeline_result['error'] = str(e)
        pipeline_result['status'] = 'failed'
    
    return pipeline_result

# Test complete Phase 5 pipeline
print("=== Testing Complete Phase 5 Pipeline ===")

# Test without personalization
phase5_result_basic = phase5_pipeline(test_phase4_output)
print(f"Basic pipeline status: {phase5_result_basic['status']}")

# Test with personalization
test_user_profile = {
    'education_level': 'college',
    'health_literacy': 'medium',
    'misinformation_vulnerability': 'high'
}
phase5_result_personalized = phase5_pipeline(test_phase4_output, test_user_profile)

# Print summary
final_output = phase5_result_personalized['final_output']
print(f"\nPersonalized pipeline status: {phase5_result_personalized['status']}")
print(f"\nFinal output summary:")
print(f"  Claim: {final_output['claim'][:60]}...")
print(f"  Verdict: {final_output['verdict_summary']}")
print(f"  Manipulation chips: {final_output['manipulation_chips']['count']}")
print(f"  Manipulation level: {final_output['manipulation_chips']['manipulation_level']}")
print(f"  Misleading factors: {len(final_output['misleading_factors'])}")
print(f"  Learning points: {final_output['education_summary']['learning_points_count']}")
print(f"  Reading time: {final_output['education_summary']['reading_time']} seconds")
print(f"  Personalized: {final_output['education_summary']['personalized']}")
print(f"  Ready for Phase 6: {final_output['ready_for_phase6']}")

# Show manipulation chips
print(f"\nDetected manipulation techniques:")
for chip in final_output['manipulation_chips']['chips']:
    print(f"  - {chip['display_text']}: {chip['tooltip']}")

# Show prebunk card summary
card = final_output['prebunk_card']
print(f"\nPrebunk card: {card['title']}")
print(f"  Main takeaway: {card['main_takeaway'][:80]}...")
print(f"  Learning points: {len(card['learning_points'])}")
print(f"  Related topics: {', '.join(card['related_topics'])}")